In [1]:
# Install necessary libraries
# !pip install cdsapi xarray matplotlib netCDF4

import cdsapi
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
# Initialize the CDS API client
c = cdsapi.Client()

# Define the Alberta bounding box (North, West, South, East)
# bbox = [60, -120, 50, -110]  # Alberta, Canada
bbox = [83, -170, 24, -50]  # North America
pressure_levels = ['250', '500', '850', '1000'] 
start_year = 1940
end_year = 2023

# Path to save the downloaded NetCDF file
dir = Path(r'D:\UCalgary_Lectures\GEOG_683\Data_workspace')
output_file = dir / 'NAmerica_geopotential_1940_2023.nc'

# Step 1: Download the data using the CDS API
if not output_file.exists():
    print("Downloading ERA5 data from {0} to {1}...".format(start_year, end_year))
    c.retrieve(
        'reanalysis-era5-pressure-levels-monthly-means',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': ['u_component_of_wind', 'v_component_of_wind', 'geopotential', 'temperature'],  
            'pressure_level': pressure_levels,
            'year': [str(year) for year in range(start_year, end_year+1)],
            'month': [f'{month:02d}' for month in range(1, 13)],
            'time': ['12:00'],
            'area': bbox,
        },
        str(output_file)
    )
    print("Download complete.")
else:
    print(f"File {output_file} already exists. Skipping download.")

In [ ]:
import numpy as np

# Step 2: Load and process the data using xarray
if output_file.exists():
    # Step 1: Load the data using xarray
    data = xr.open_dataset(output_file)

    # Step 2: Extract the relevant variables at 250mb
    u_250mb = data['u'].sel(pressure_level=250)  # Zonal wind (east-west)
    v_250mb = data['v'].sel(pressure_level=250)  # Meridional wind (north-south)
    geopotential_250mb = data['z'].sel(pressure_level=250) / 9.81  # Geopotential height (converted to meters)

    # Step 3: Calculate the mean over the entire period
    u_mean = u_250mb.mean(dim='date')  # Mean of zonal wind over all months
    v_mean = v_250mb.mean(dim='date')  # Mean of meridional wind over all months
    geo_height_mean = geopotential_250mb.mean(dim='date')  # Mean geopotential height

    # Step 4: Calculate the wind speed (magnitude)
    wind_speed = np.sqrt(u_mean**2 + v_mean**2)  # Wind speed is the magnitude of (u, v)

    # Flatten the plot by setting the contour levels
    geo_height_std = geopotential_250mb.std(dim='date')  # Standard deviation of geopotential height
    levels_geo = np.linspace((geo_height_mean - 2 * geo_height_std).min(), (geo_height_mean + 2 * geo_height_std).max(), 100)
    levels_wind = np.linspace(wind_speed.min(), wind_speed.max(), 40)  # Levels for wind speed

    # Step 5: Plot the geopotential height map and wind speed as colored contours
    plt.figure(figsize=(10, 6))

    # Plot the geopotential height as a filled contour map
    contour_geo = plt.contourf(geo_height_mean.longitude, geo_height_mean.latitude, geo_height_mean, levels=levels_geo, cmap='BuPu')
    plt.colorbar(contour_geo, label='Geopotential Height (meters)')

    # Plot the wind speed as colored contours
    # 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 
    # 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 
    # 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 
    # 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 
    # 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 
    # 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 
    # 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 
    # 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 
    # 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 
    # 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 
    # 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 
    # 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 
    # 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 
    # 'winter', 'winter_r'

    contour_wind = plt.contourf(u_mean.longitude, u_mean.latitude, wind_speed, levels=levels_wind, cmap='jet', alpha=0.6)
    plt.colorbar(contour_wind, label='Wind Speed (m/s)')

    # Add title and labels
    plt.title('Monthly Averaged Wind Flow Patterns at 250mb (2013-2023)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    # Show the plot
    plt.grid(True)
    plt.show()
else:
    print(f"File {output_file} does not exist. Please check the path or download step.")
